## A simple swarm

In [1]:
%load_ext autoreload 
%autoreload 2
import sys
sys.tracebacklimit = 0

In [2]:
from core.agent import *
from core.world import * 
from core.map import *
import numpy as np
from core.render import * 

pygame-ce 2.4.1 (SDL 2.28.5, Python 3.11.9)


In [3]:
def initialize_swarm(world : World):
    swarm = [Agent() for i in range(1000)]
    for agent in swarm:
        world.add_agent(agent)
    swarm = initialize_positions_randomly(world, swarm)
    

In [4]:
world = World(dims = (100, 100),
              swarm_initialzier= initialize_swarm,
              resource_generator= RandomMapGenerator(),
              energy_model=EnergyModel()
              )
world.reset()

In [14]:
# Test movements by doing random actions
def update():
    world.update()
    # Comms protocol (receiving)
    for agent in world.agents: 
        received_messages = agent.get_messages()
        for msg in received_messages: 
            agent.add_relation(msg.sender, msg.message)
        agent.clear_messages()

    for agent in world.agents:
        action_choice = np.random.randint(0, 4)
        choice = np.random.randint(1, 5)

        match (action_choice): 
            case 1: agent.move(choice)
            case 2: agent.pick_up(choice)
            case 3: agent.put_down(choice)
    
    # Comms protocol (sending )
    for agent in world.agents:
        neighbors = agent.agents_in_range 

        # Send agents
        if len (neighbors) > 0:
            choice = np.random.choice(neighbors)

            tgt_agent = world.get_agent(choice)
            agent.talk_to(tgt_agent)

update()

SyntaxError: invalid syntax (4136152055.py, line 13)

In [6]:
render_world(world, (800, 800), update_fn=update, delay_s=0)

## Verify certain things about the simulation

In [7]:
swarm = world.agents

In [8]:
# Validate the observation space provided is sensible
obs = swarm[0].local_observation

print(obs.nearby_agents, obs.nearby_agents.shape)
print(obs.resource_types, obs.resource_types.shape)

[[  0   0   0   0   0 493   0]
 [  0 345   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0 157   0   0   0   0]
 [  0   0   0   0   0 713   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0 949   0   0]] (7, 7)
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]] (7, 7)


In [9]:
# Validate that all swarm agents landed in different positions
positions = set()
for agent in swarm:
    pos = agent.current_position
    positions.add((pos[0], pos[1]))

print(len(positions), len(world.agents))

1000 1000


In [10]:
# Check for resources
rsrc_qty = world._resource_map._resource_quantity_map
qtys = []

for i in range(rsrc_qty.shape[0]):
    for j in range(rsrc_qty.shape[1]):
        if rsrc_qty[i, j] > 0:
            qtys.append(rsrc_qty[i, j])

print(qtys)

[5.362267890186665, 2.5076911069305328, 2.1118076272991892, 2.47637957908045, 3.058396235206019, 6.797806113829335, 6.177162438663717, 4.70589714163713, 0.21809431147672065, 2.9313841914315732, 8.298775406910156, 4.890214145067446, 4.687772219420258, 6.349467890239881, 4.95482087327405, 3.0303439985740845, 2.8285926337241407, 3.1203821291662965, 5.235385791431114, 3.8747259298865915, 3.5948283442320967, 4.70950180179118, 6.094032256256625, 3.5980246096604356, 8.107794716647822, 4.10700979149844, 4.507481464975366, 7.6770324050900935, 6.3534614703672005, 3.509150925967269, 8.277413199843393, 0.3554742055304265, 1.3459933774365718, 6.1565359563957545, 6.500236485731293, 3.982576408567114, 7.275492396042088, 3.821297084065387, 6.303077799541276, 2.5715630871046384, 3.446567500507718, 5.274180525151952, 7.218975574452898, 6.531069859953942, 0.7137817739441106, 2.6660524575960203, 3.899697790146252, 5.225167325584379, 5.796785884553858, 6.38429798373932, 6.64128896904788, 5.313425205540721,

In [11]:
# Check for agent state
state = swarm[0]._current_state.inventory
print(state)


[Resource(type=3, quantity=1.0)]


# Perftest

In [12]:
import cProfile
def stress_test():
    for _ in range(0, 1000):
        update()

In [13]:
world.reset()
cProfile.run('stress_test()', sort = 'time')

         77020562 function calls in 36.520 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  2000000    3.002    0.000    3.002    0.000 {method 'randint' of 'numpy.random.mtrand.RandomState' objects}
   989825    2.928    0.000    6.873    0.000 {method 'choice' of 'numpy.random.mtrand.RandomState' objects}
     1000    2.640    0.003   36.706    0.037 2985805406.py:2(update)
  1000000    1.774    0.000    2.235    0.000 observation.py:20(neighbors)
     1000    1.762    0.002    4.005    0.004 world.py:88(_update_movement)
  1000000    1.592    0.000    2.790    0.000 world.py:129(_get_nearby_agents)
  1000000    1.566    0.000    2.733    0.000 world.py:149(_get_nearby_resources)
  6535711    1.431    0.000    1.431    0.000 {method 'copy' of 'numpy.ndarray' objects}
  1239814    1.422    0.000    1.422    0.000 {method 'reduce' of 'numpy.ufunc' objects}
  1239814    1.323    0.000    3.721    0.000 fromnumeric.py:71(_wr